<a href="https://colab.research.google.com/github/FernandaSzAdr/Redes-Neurais--UFRPE-2019.1/blob/master/MLP/codigos/Backpropagation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/FernandaSzAdr/Redes-Neurais--UFRPE-2019.1

Cloning into 'Redes-Neurais--UFRPE-2019.1'...
remote: Enumerating objects: 120, done.
remote: Counting objects: 100% (120/120), done.
remote: Compressing objects: 100% (101/101), done.
remote: Total 120 (delta 23), reused 30 (delta 2), pack-reused 0
Receiving objects: 100% (120/120), 21.68 MiB | 27.47 MiB/s, done.
Resolving deltas: 100% (23/23), done.


##Multipayer Perceptron (MLP)

Rede Neural baseado no algoritmo de gradiente descendente.  
Os gradientes são calculados usando backpropagation. 

Para mais detalhes, ver os capitulos 13 a 16 do livro no site:

http://deeplearningbook.com.br/

In [2]:
!pip install ipdb
import random
import numpy as np

  Stored in directory: /root/.cache/pip/wheels/59/24/91/695211bd228d40fb22dff0ce3f05ba41ab724ab771736233f3
Successfully built ipdb


A entrada é uma lista (`sizes`) contém o número de neurônios nas respectivas camadas da rede. Por exemplo, se a lista for [2, 3, 1] então será uma rede de três camadas, com o primeira camada contendo 2 neurônios, a segunda camada 3 neurônios, e a terceira camada 1 neurônio. Os bias e pesos para a rede são inicializados aleatoriamente, usando uma distribuição Gaussiana com média 0 e variância 1. Note que a primeira camada é assumida como uma camada de entrada, e por convenção não definimos nenhum bias para esses neurônios, pois os bias são usados na computação das saídas das camadas posteriores.


In [0]:
# Classe Network
class Network(object):

    def __init__(self, sizes):
        self.num_layers = len(sizes)
        self.sizes = sizes
        self.biases = [np.random.randn(y, 1) for y in sizes[1:]]
        self.weights = [np.random.randn(y, x) for x, y in zip(sizes[:-1], sizes[1:])]
        self.true_predic = {}

    def feedforward(self, a):
        """Retorna a saída da rede se `a` for entrada."""
        for b, w in zip(self.biases, self.weights):
            a = relu(np.dot(w, a)+b)
        return a

    def SGD(self, training_data, epochs, mini_batch_size, eta, test_data=None):
        """Treinar a rede neural usando o algoritmo mini batch com gradiente descendente. 
         A entrada é uma lista de tuplas
         `(x, y)` representando as entradas de treinamento e as
         saídas. Os outros parâmetros não opcionais são
         auto-explicativos. Se `test_data` for fornecido, então a
         rede será avaliada em relação aos dados do teste após cada
         época e progresso parcial impresso. Isso é útil para
         acompanhar o progresso, mas retarda as coisas substancialmente."""
        
        eta_inicial = eta
        training_data = list(training_data)
        n = len(training_data)

        if test_data:
            n_test = len(test_data)

        for j in range(epochs):
            random.shuffle(training_data)
            mini_batches = [training_data[k:k+mini_batch_size] for k in range(0, n, mini_batch_size)]
            
            for mini_batch in mini_batches:
                self.update_mini_batch(mini_batch, eta)
            
            if test_data:
                predic = self.evaluate(test_data)
                self.true_predic[j] = predic
                print("Epoch {} : {} / {}".format(j,predic,n_test));
            else:
                print("Epoch {} finalizada".format(j))
            
            eta = atualiza_taxa_aprend(eta_inicial, j, epochs)
        
        return self.true_predic
    def update_mini_batch(self, mini_batch, eta):
        """Atualiza os pesos e bias da rede aplicando
         a descida do gradiente usando backpropagation para um único mini lote.
         O `mini_batch` é uma lista de tuplas `(x, y)`, e `eta` é a taxa de aprendizado."""

        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        
        for x, y in mini_batch:
            delta_nabla_b, delta_nabla_w = self.backprop(x, y)
            nabla_b = [nb+dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]
            nabla_w = [nw+dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]
        
        self.weights = [w-(eta/len(mini_batch))*nw for w, nw in zip(self.weights, nabla_w)]
        self.biases = [b-(eta/len(mini_batch))*nb for b, nb in zip(self.biases, nabla_b)]

    def backprop(self, x, y):
        """Retorna uma tupla `(nabla_b, nabla_w)` representando o
         gradiente para a função de custo C_x. `nabla_b` e
         `nabla_w` são listas de camadas de matrizes numpy, semelhantes
         a `self.biases` e `self.weights`."""

        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        
        # Feedforward
        activation = x

        # Lista para armazenar todas as ativações, camada por camada
        activations = [x] 

        # Lista para armazenar todos os vetores z, camada por camada
        zs = [] 

        for b, w in zip(self.biases, self.weights):
            z = np.dot(w, activation)+b
            zs.append(z)
            activation = relu(z)
            activations.append(activation)
        
        # Backward pass
        delta = self.cost_derivative(activations[-1], y) * relu_prime(zs[-1])
        nabla_b[-1] = delta
        nabla_w[-1] = np.dot(delta, activations[-2].transpose())
        
        # Aqui, l = 1 significa a última camada de neurônios, l = 2 é a segunda e assim por diante. 
        for l in range(2, self.num_layers):
            z = zs[-l]
            sp = relu_prime(z)
            delta = np.dot(self.weights[-l+1].transpose(), delta) * sp
            nabla_b[-l] = delta
            nabla_w[-l] = np.dot(delta, activations[-l-1].transpose())
        return (nabla_b, nabla_w)

    def evaluate(self, test_data):
        """Retorna o número de entradas de teste para as quais a rede neural 
         produz o resultado correto. Note que a saída da rede neural
         é considerada o índice de qualquer que seja
         neurônio na camada final que tenha a maior ativação."""

        test_results = [(np.argmax(self.feedforward(x)), np.argmax(y)) for (x, y) in test_data]
        return sum(int(x == y) for (x, y) in test_results)

    def cost_derivative(self, output_activations, y):
        """Retorna o vetor das derivadas parciais."""
        return (output_activations-y)

# Função de Ativação Sigmóide
def sigmoid(z):
    return 1.0/(1.0+np.exp(-z))
  
def relu(z):
    return np.array([[max(0, i[0])] for i in z])

# Função para retornar as derivadas da função Sigmóide
def sigmoid_prime(z):
    return sigmoid(z)*(1-sigmoid(z))

def relu_prime(z):
    return relu(z)*(1-relu(z))

def atualiza_taxa_aprend(taxa_inicial, epoch, n_epochs):
    return taxa_inicial * np.exp(-epoch / n_epochs)


Como exemplo, essa mesma rede será executada na base de dados MNIST. O codigo abaixo carrega a base de dados.

In [0]:
# Carregar o dataset MNIST

# Imports
import pickle
import gzip
import numpy as np

def load_data():
    f = gzip.open('Redes-Neurais--UFRPE-2019.1/MLP/codigos/mnist.pkl.gz', 'rb')
    training_data, validation_data, test_data = pickle.load(f, encoding="latin1")
    f.close()
    return (training_data, validation_data, test_data)

def load_data_wrapper():
    tr_d, va_d, te_d = load_data()
    training_inputs = [np.reshape(x, (784, 1)) for x in tr_d[0]]
    training_results = [vectorized_result(y) for y in tr_d[1]]
    training_data = zip(training_inputs, training_results)
    validation_inputs = [np.reshape(x, (784, 1)) for x in va_d[0]]
    validation_data = zip(validation_inputs, va_d[1])
    test_inputs = [np.reshape(x, (784, 1)) for x in te_d[0]]
    test_data = zip(test_inputs, te_d[1])
    return (training_data, validation_data, test_data)

def vectorized_result(j):
    e = np.zeros((10, 1))
    e[j] = 1.0
    return e

def all_data():
  training_data, validation_data, test_data = load_data_wrapper()

  training_data = list(training_data)
  validation_data = list(validation_data)
  test_data = list(test_data)

  return training_data + validation_data + test_data

#Executa a rede neural

Parâmetros de rede:
         2º param é contagem de épocas
         3º param é tamanho do lote
         4º param é a taxa de aprendizado (eta)

##Mini-Projeto

1) Realizar avaliaçao usando validação cruzada

2) Modificar a taxa de aprendizagem pra diminuir com o tempo

3) Realizar avaliações com a validação cruzada modificando os seguintes parâmetros:
     
     a) Taxa de aprendizagem: 0.5, 0.1 e 0.001
     b) Função de ativação RELU
     c) 3 diferentes configurações de neurônios na camada  intermediária


Data de Entrega: 23/04/2019

A partir dos melhores resultados obtidos na etapa anterior, resolver as seguintes questões:

4) Implementar as regularizações L1 e L2

5) Implementar o Momento

6) Comparar os experimentos e explicar o que achou dos resultados.

Data de Entrega: 30/04/2019




In [0]:
import pandas as pd
from sklearn.model_selection import KFold

In [0]:
data = all_data()

## Rede 1 -> Configuração 784-30-10

### Taxa de aprendizagem - 0.5

In [0]:
net = Network([784, 30, 10])
acertos_dict = {}
kf = KFold(n_splits=10)
at = 0.5

for i, index in enumerate(kf.split(data)):
  train_index = index[0]
  test_index = index[1]
  
  k = "K {}".format(i)
  
  print(k)
  training_data = data[train_index[0]:train_index[-1]+1]
  testing_data = data[test_index[0]:test_index[-1]+1]
  
  result = net.SGD(training_data, 15, 10, at,test_data=testing_data)
  acertos_dict[k] = sum(result.values())

K 0


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:91: RuntimeWarning: overflow encountered in multiply
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:99: RuntimeWarning: invalid value encountered in multiply
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:129: RuntimeWarning: overflow encountered in multiply


In [0]:
teste1 = pd.DataFrame({
    'K': np.arange(kf.get_n_splits(data)),
    'Acertos': list(acertos_dict.values())
})

teste1

In [0]:
teste1.groupby('Acertos').mean()